In [4]:
from fiat_toolbox.well_being import Household
import matplotlib.pyplot as plt

Let's assume we have a household that has a house with a building structure value of '**k_str**' that is damaged by a flood leading to a repair costs '**v**', representing the ratio of the structure value (i.e., v=0.7 describes repair costs that are 70% of the building structure value). Let's assume that the annual consumption of the household '**c0**' is equal to the annual income of the household. And that the average consumption of the area of interest is '**c_avg**' (represented by the average income)

In [ ]:
v = 0.7 # fraction of damage to the building
k_str = 30000 # building value
c0 = 48000 # annual income
c_avg = 48000 # average annual income

We can define some economic indicators to assess specific type of losses.

In [ ]:
pi = 0.15 # productivity of capital index
eta = 1.5 # elasticity of the marginal utility
rho = 0.06 # discount rate
t_max = 10 # recovery time in years
dt = 1/52 # time step in years
currency = "$"

Then we can use the Household class to define a household with these characteristics.

In [ ]:
# Create a WellBeing object and optimize the lambda value
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, pi=pi, eta=eta, rho=rho, dt=dt, t_max=t_max, currency=currency)
household

We could have provided already a reconstruction rate with the argument "l". Since this is normally not known in advance, we can actually calculate the recovery rate that would optimize the utility loss of the household using the "opt_lambda" method. We can visualize the optimization with the "plot_opt_lambda" method.

In [ ]:
household.opt_lambda(method="trapezoid")
household.plot_opt_lambda(x_type="time");

In [ ]:
# Create a WellBeing object and optimize the lambda value
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1/52)
household.opt_lambda(method="trapezoid")
l = household.l_opt["lambda"]
z1 = household.l_opt["Utility Loss"]

household.opt_lambda(method="quad")
z2 = household.l_opt["Utility Loss"]

# Plot the results
plt.plot(l, z1, label='trapezoid Method')
plt.plot(l, z2, label='quad Method')
plt.xlabel('Lambda')
plt.ylabel('Utility Loss')
plt.legend()
plt.show()

In [ ]:
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1/54)
household.opt_lambda(method="trapezoid")
household.get_losses("trapezoid")

In [ ]:
household.plot_consumption()